In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai import *
from fastai.tabular import *

# Rossmann

## Data Preparation

To create the feature-engineered train_clean and test_clean from the Kaggle competition data, run `rossman_data_clean.ipynb`. One important step that deals with time series is this
    
```python
add_datepart(train, \"Date\", drop=False)
add_datepart(test, \"Date\", drop=False)
```

In [1]:
path = Path('data/rossmann/')
train_df = pd.read_pickle(path/'train_clean')

NameError: name 'Path' is not defined